In [0]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensornets as nets
import cv2
import numpy as np
import time

Instructions for updating:
non-resource variables are not supported in the long term


# Using Trained YOLOv3 Model

In [0]:

inputs = tf.placeholder(tf.float32, [None, 416, 416, 3])
model = nets.YOLOv3COCO(inputs, nets.Darknet19)

classes={'0':'person','1':'bicycle','2':'car','3':'bike','5':'bus','7':'truck'}
list_of_classes=[0,1,2,3,5,7]
 

Instructions for updating:
Please use `layer.__call__` method instead.


# Predicting for videos

In [0]:
with tf.Session() as sess:
      sess.run(model.pretrained())
    
      cap = cv2.VideoCapture("shinjuku.mp4")
      # for video writing
      frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                            cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
      # fourcc = cv2.VideoWriter_fourcc(*'X264')
      fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
      fps = cap.get(cv2.CAP_PROP_FPS)
      out = cv2.VideoWriter('detections.mp4', fourcc, fps,
                                    (int(frame_size[0]), int(frame_size[1])))
      
      while(cap.isOpened()):
          # print("cap open")
          ret, frame = cap.read()
          try:
            img=cv2.resize(frame,(416,416))
          except Exception as e:
            break
            
          imge=np.array(img).reshape(-1,416,416,3)
          start_time=time.time()
          preds = sess.run(model.preds, {inputs: model.preprocess(imge)})
      
          print("--- %s seconds ---" % (time.time() - start_time)) 
          boxes = model.get_boxes(preds, imge.shape[1:3])
          # cv2.namedWindow('image',cv2.WINDOW_NORMAL)

          # cv2.resizeWindow('image', 700,700)
          #print("--- %s seconds ---" % (time.time() - start_time)) 
          boxes1=np.array(boxes)
          for j in list_of_classes:
              count =0
              if str(j) in classes:
                  lab=classes[str(j)]
              if len(boxes1) !=0:
                  
                  
                  for i in range(len(boxes1[j])):
                      box=boxes1[j][i] 
                      
                      if boxes1[j][i][4]>=.40:
                          
                              
                          count += 1    
  
                          cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),(0,255,0),1)
                          cv2.putText(img, lab, (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), lineType=cv2.LINE_AA)
                          
              # print(lab,": ",count)
          
          out.write(cv2.resize(img,(640,360) ))  
          # cv2.imshow("image",img)  
          if cv2.waitKey(1) & 0xFF == ord('q'):
              break          




  cap.release()
  out.release()
  cv2.destroyAllWindows()   

Streaming output truncated to the last 5000 lines.
--- 0.031682491302490234 seconds ---
--- 0.031502485275268555 seconds ---
--- 0.03195619583129883 seconds ---
--- 0.03183770179748535 seconds ---
--- 0.03105330467224121 seconds ---
--- 0.031969547271728516 seconds ---
--- 0.03164029121398926 seconds ---
--- 0.03196215629577637 seconds ---
--- 0.03185844421386719 seconds ---
--- 0.03250885009765625 seconds ---
--- 0.032292842864990234 seconds ---
--- 0.032379865646362305 seconds ---
--- 0.031716108322143555 seconds ---
--- 0.03156566619873047 seconds ---
--- 0.03109574317932129 seconds ---
--- 0.031781673431396484 seconds ---
--- 0.031214237213134766 seconds ---
--- 0.03232979774475098 seconds ---
--- 0.03095722198486328 seconds ---
--- 0.03091263771057129 seconds ---
--- 0.030982494354248047 seconds ---
--- 0.031942129135131836 seconds ---
--- 0.03139138221740723 seconds ---
--- 0.031222820281982422 seconds ---
--- 0.03304004669189453 seconds ---
--- 0.031545162200927734 seconds ---
-

# Predicting for Images

In [0]:
# list out images that need to be predicted
input_names =[]

In [0]:
def draw_boxes(img_names, boxes_dicts, class_names, model_size):
    """Draws detected boxes.
    Args:
        img_names: A list of input images names.
        boxes_dict: A class-to-boxes dictionary.
        class_names: A class names list.
        model_size: The input size of the model.
    Returns:
        None.
    """
    colors = ((np.array(color_palette("hls", 80)) * 255)).astype(np.uint8)
    for num, img_name, boxes_dict in zip(range(len(img_names)), img_names,
                                         boxes_dicts):
        img = Image.open(img_name)
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(font='./data/fonts/futur.ttf',
                                  size=(img.size[0] + img.size[1]) // 100)
        resize_factor = \
            (img.size[0] / model_size[0], img.size[1] / model_size[1])
        for cls in range(len(class_names)):
            boxes = boxes_dict[cls]
            if np.size(boxes) != 0:
                color = colors[cls]
                for box in boxes:
                    xy, confidence = box[:4], box[4]
                    xy = [xy[i] * resize_factor[i % 2] for i in range(4)]
                    x0, y0 = xy[0], xy[1]
                    thickness = (img.size[0] + img.size[1]) // 200
                    for t in np.linspace(0, 1, thickness):
                        xy[0], xy[1] = xy[0] + t, xy[1] + t
                        xy[2], xy[3] = xy[2] - t, xy[3] - t
                        draw.rectangle(xy, outline=tuple(color))
                    text = '{} {:.1f}%'.format(class_names[cls],
                                               confidence * 100)
                    text_size = draw.textsize(text, font=font)
                    draw.rectangle(
                        [x0, y0 - text_size[1], x0 + text_size[0], y0],
                        fill=tuple(color))
                    draw.text((x0, y0 - text_size[1]), text, fill='black',
                              font=font)
                    print('{} {:.2f}%'.format(class_names[cls],
                                              confidence * 100))

        rgb_img = img.convert('RGB')

        rgb_img.save('./detections/detection_' + str(num + 1) + '.jpg')


In [0]:
inputs = tf.placeholder(tf.float32, [None, 416, 416, 3])


img = nets.utils.load_img('# name of image file')

with tf.Session() as sess:
    sess.run(model.pretrained())
    preds = sess.run(model.preds, {inputs: img})
    boxes = model.get_boxes(preds, img.shape[1:3])

# draw_boxes(img, preds, class_names, [None, 416, 416, 3])